In [1]:
import pandas as pd
import numpy as np
import mysql.connector as connector

import warnings
warnings.filterwarnings("ignore")

### Prepare dataframes tables

In [2]:
df = pd.read_excel('LittleLemon_data.xlsx')

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row Number     21000 non-null  int64         
 1   Order ID       21000 non-null  object        
 2   Order Date     21000 non-null  datetime64[ns]
 3   Delivery Date  21000 non-null  datetime64[ns]
 4   Customer ID    21000 non-null  object        
 5   Customer Name  21000 non-null  object        
 6   City           21000 non-null  object        
 7   Country        21000 non-null  object        
 8   Postal Code    21000 non-null  object        
 9   Country Code   20958 non-null  object        
 10   Cost          21000 non-null  float64       
 11  Sales          21000 non-null  float64       
 12  Quantity       21000 non-null  int64         
 13  Discount       21000 non-null  float64       
 14  Delivery Cost  21000 non-null  float64       
 15  Course Name    2100

In [3]:
df_cuisine = df['Cuisine Name'].dropna().drop_duplicates().reset_index(drop=True)
df_cuisine.index = df_cuisine.index + 1
df_cuisine = pd.DataFrame(df_cuisine)
df_cuisine.columns = ['Name']

df_cuisine

,Name
1,Greek
2,Italian
3,Turkish


In [4]:
df_menuitem = pd.concat([ df['Starter Name'], df['Course Name'], df['Drink'], df['Desert Name'] ])
df_menuitem = df_menuitem.dropna().drop_duplicates().reset_index(drop=True)
df_menuitem.index = df_menuitem.index + 1
df_menuitem = pd.DataFrame(df_menuitem)
df_menuitem.columns = ['ItemName']
df_menuitem['ItemDescription'] = 'Placeholder <' + df_menuitem['ItemName'] + '>'

df_menuitem

,ItemName,ItemDescription
1,Olives,Placeholder <Olives>
2,Flatbread,Placeholder <Flatbread>
3,Minestrone,Placeholder <Minestrone>
4,Tomato bread,Placeholder <Tomato bread>
5,Falafel,Placeholder <Falafel>
6,Hummus,Placeholder <Hummus>
7,Greek salad,Placeholder <Greek salad>
8,Bean soup,Placeholder <Bean soup>
9,Pizza,Placeholder <Pizza>
10,Carbonara,Placeholder <Carbonara>


In [5]:
df_menu = df[['Cuisine Name', 'Starter Name', 'Course Name', 'Drink', 'Desert Name']]
df_menu = df_menu.dropna().drop_duplicates().reset_index(drop=True)
df_menu.index = df_menu.index + 1
df_menu.columns = ['CuisineID', 'StarterID', 'CourseID', 'DrinkID', 'DessertID']
df_menu['CuisineID'] = df_menu['CuisineID'].replace(df_cuisine['Name'].values, df_cuisine.index)
df_menu['StarterID'] = df_menu['StarterID'].replace(df_menuitem['ItemName'].values, df_menuitem.index)
df_menu['CourseID'] = df_menu['CourseID'].replace(df_menuitem['ItemName'].values, df_menuitem.index)
df_menu['DrinkID'] = df_menu['DrinkID'].replace(df_menuitem['ItemName'].values, df_menuitem.index)
df_menu['DessertID'] = df_menu['DessertID'].replace(df_menuitem['ItemName'].values, df_menuitem.index)

df_menu

,CuisineID,StarterID,CourseID,DrinkID,DessertID
1,1,1,7,13,19
2,2,2,8,14,20
3,2,3,9,15,21
4,3,4,10,16,22
5,1,5,11,17,23
...,...,...,...,...,...
68,2,2,7,13,19
69,3,3,8,14,20
70,1,4,9,15,21
71,2,5,10,16,22


In [6]:
df_address = df[['City', 'Country', 'Postal Code']]
df_address = df_address.dropna().drop_duplicates()
df_address = df_address[df_address['Country'] == 'United States'].reset_index(drop=True)
df_address.index = df_address.index + 1
df_address.columns = ['City', 'Country', 'PostalCode']
df_address['City'] = df_address['City'].str.encode('utf-8').str.decode('utf-8')
df_address.insert(0, 'StreetName', 'Placeholder <' + df_address['City'] + '>')

df_address

,StreetName,City,Country,PostalCode
1,Placeholder <Shawnee Mission>,Shawnee Mission,United States,993-0039
2,Placeholder <Provo>,Provo,United States,359 01
3,Placeholder <Reno>,Reno,United States,3810-771
4,Placeholder <Sacramento>,Sacramento,United States,-6449616.666667
5,Placeholder <Allentown>,Allentown,United States,17062310
...,...,...,...,...
647,Placeholder <Phoenix>,Phoenix,United States,-589504480.666667
648,Placeholder <Fairbanks>,Fairbanks,United States,1581 01
649,Placeholder <Pasadena>,Pasadena,United States,1346611895
650,Placeholder <Santa Cruz>,Santa Cruz,United States,34850


In [7]:
np.random.seed(0)

order_subset_indices = df['Delivery Date'].sample(2000).index.sort_values()
indices_delivery = np.sort(np.random.choice(order_subset_indices, size=1000, replace=False))
indices_booking = order_subset_indices.drop(indices_delivery).sort_values().values

In [8]:
df_delivery = df[['Delivery Date']].loc[indices_delivery].reset_index(drop=True)
df_delivery.index = df_delivery.index + 1
df_delivery['Status'] = 'Delivered'
df_delivery['AddressID'] = np.random.choice(df_address.index, size=len(df_delivery), replace=True)

df_delivery

,Delivery Date,Status,AddressID
1,2022-01-07,Delivered,562
2,2019-12-16,Delivered,334
3,2021-06-08,Delivered,319
4,2022-05-29,Delivered,552
5,2020-11-04,Delivered,643
...,...,...,...
996,2020-08-10,Delivered,111
997,2022-12-03,Delivered,205
998,2021-01-31,Delivered,372
999,2021-01-01,Delivered,560


In [9]:
df_booking = df[['Delivery Date']].loc[indices_booking].reset_index(drop=True)
df_booking.index = df_booking.index + 1
df_booking.columns = ['BookingDate']
df_booking['TableNumber'] = np.random.randint(1, 20, size=len(df_booking))

df_booking

,BookingDate,TableNumber
1,2020-12-27,17
2,2022-10-14,5
3,2022-04-06,15
4,2021-12-28,4
5,2022-09-01,8
...,...,...
996,2020-05-06,17
997,2021-04-27,12
998,2022-02-26,9
999,2019-12-15,17


In [10]:
df_customer = df[['Customer Name', 'Customer ID']]
df_customer = df_customer.dropna().drop_duplicates().sort_values('Customer Name').reset_index(drop=True)
df_customer.index = df_customer.index + 1
df_customer.columns = ['FullName', 'ContactNumber']
df_customer['Email'] = 'Placeholder <' + df_customer['FullName'] + '>'

df_customer

,FullName,ContactNumber,Email
1,Aaron Cromley,03-350-0766,Placeholder <Aaron Cromley>
2,Abigail Struis,10-972-8681,Placeholder <Abigail Struis>
3,Adah Adamczewski,89-643-5055,Placeholder <Adah Adamczewski>
4,Adaline Reichartz,06-093-4453,Placeholder <Adaline Reichartz>
5,Adamo Rumens,58-684-1818,Placeholder <Adamo Rumens>
...,...,...,...
996,Zita Coldbath,14-149-6520,Placeholder <Zita Coldbath>
997,Zolly Heinzel,79-230-2093,Placeholder <Zolly Heinzel>
998,Zora Hanalan,91-616-5387,Placeholder <Zora Hanalan>
999,Zorana Diggin,06-346-0589,Placeholder <Zorana Diggin>


In [11]:
df_staff = pd.DataFrame(df_customer['FullName'].sample(20).reset_index(drop=True))
df_staff.index = df_staff.index + 1
df_staff['Role'] = np.random.choice(['Chef', 'Waiter', 'Manager'], size=len(df_staff), replace=True)
df_staff['Salary'] = np.random.randint(20000, 60000, size=len(df_staff))

df_staff

,FullName,Role,Salary
1,Yankee Penright,Waiter,23262
2,Cynthea O'Hickee,Chef,45713
3,Shanan Mingay,Waiter,35876
4,Marsiella Duprey,Waiter,47568
5,Mady McMennum,Manager,27933
6,Jordan Donner,Chef,24399
7,Verena Garman,Chef,54420
8,Igor Durrand,Manager,21320
9,Pansie Alldis,Manager,38978
10,Sibley Rodder,Waiter,52442


In [12]:
df_order_ord = df[['Order Date', 'Quantity', ' Cost']].iloc[indices_booking]
df_order_ord.columns = ['OrderDate', 'Quantity', 'TotalCost']
df_order_ord['OrderType'] = 'Booking'
df_order_ord['BookingID'] = df_booking.index

df_order_del = df[['Order Date', 'Quantity', ' Cost']].iloc[indices_delivery]
df_order_del.columns = ['OrderDate', 'Quantity', 'TotalCost']
df_order_del['OrderType'] = 'Delivery'
df_order_del['DeliveryID'] = df_delivery.index

df_order = pd.concat([df_order_ord, df_order_del]).sort_index().reset_index(drop=True)
df_order.index = df_order.index + 1
df_order['MenuID'] = np.random.choice(df_menu.index, size=len(df_order), replace=True)
df_order['CustomerID'] = np.random.choice(df_customer.index, size=len(df_order), replace=True)
df_order['StaffID'] = np.random.choice(df_staff.index, size=len(df_order), replace=True)

df_order

,OrderDate,Quantity,TotalCost,OrderType,BookingID,DeliveryID,MenuID,CustomerID,StaffID
1,2021-10-09,3,225.00,Booking,1.0,NaN,52,458,1
2,2020-12-03,2,93.13,Delivery,NaN,1.0,17,652,20
3,2019-09-04,2,52.08,Booking,2.0,NaN,6,454,1
4,2020-04-14,3,69.38,Booking,3.0,NaN,62,257,13
5,2022-04-05,1,44.57,Delivery,NaN,2.0,30,968,15
...,...,...,...,...,...,...,...,...,...
1996,2020-07-19,3,119.80,Booking,1000.0,NaN,47,891,16
1997,2020-06-15,2,75.00,Delivery,NaN,997.0,7,727,11
1998,2022-09-22,1,98.00,Delivery,NaN,998.0,13,155,8
1999,2021-03-01,3,93.13,Delivery,NaN,999.0,10,269,4


### Insert dataframe entries into database

In [13]:
connection = connector.connect(user="root", password="password",
                               auth_plugin="mysql_native_password")

cursor = connection.cursor()

In [14]:
cursor.execute("DROP DATABASE IF EXISTS littlelemondb;")

with open('../LittleLemonDB.sql', 'r') as file:
    cursor.execute(file.read())

In [15]:
connection = connector.connect(user="root", password="password",
                               auth_plugin="mysql_native_password")

cursor = connection.cursor()

In [16]:
for _, row in df_cuisine.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.cuisine (Name)
        VALUES ("{row.Name}");
        """
    )
    
connection.commit()

In [17]:
for _, row in df_menuitem.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.menuitem (ItemName, ItemDescription)
        VALUES ("{row.ItemName}", "{row.ItemDescription}");
        """
    )

connection.commit()

In [18]:
for _, row in df_menu.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.menu (CuisineID, StarterID, CourseID, DrinkID, DessertID)
        VALUES ({row.CuisineID}, {row.StarterID}, {row.CourseID}, {row.DrinkID}, {row.DessertID});
        """
    )

connection.commit()

In [19]:
for _, row in df_address.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.address (StreetName, City, Country, PostalCode)
        VALUES ("{row.StreetName}", "{row.City}", "{row.Country}", "{row.PostalCode}");
        """
    )

connection.commit()

In [20]:
for _, row in df_delivery.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.delivery (DeliveryDate, Status, AddressID)
        VALUES ("{row['Delivery Date']}", "{row.Status}", {row.AddressID});
        """
    )

connection.commit()

In [21]:
for _, row in df_booking.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.booking (BookingDate, TableNumber)
        VALUES ("{row.BookingDate}", {row.TableNumber});
        """
    )

connection.commit()    

In [22]:
for _, row in df_customer.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.customer (FullName, ContactNumber, Email)
        VALUES ("{row.FullName}", "{row.ContactNumber}", "{row.Email}");
        """
    )

connection.commit()    

In [23]:
for _, row in df_staff.iterrows():
    cursor.execute(
        f"""
        INSERT INTO littlelemondb.staff (FullName, Role, Salary)
        VALUES ("{row.FullName}", "{row.Role}", {row.Salary});
        """
    )

connection.commit()

In [24]:
for _, row in df_order.iterrows():
    if not np.isnan(row.BookingID):
        cursor.execute(
            f"""
            INSERT INTO littlelemondb.orders (OrderDate, Quantity, TotalCost, OrderType, BookingID, MenuID, CustomerID, StaffID)
            VALUES ("{row.OrderDate}", {row.Quantity}, {row.TotalCost}, "{row.OrderType}", {int(row.BookingID)}, {row.MenuID}, {row.CustomerID}, {row.StaffID});
            """
        )
    elif not np.isnan(row.DeliveryID):
        cursor.execute(
            f"""
            INSERT INTO littlelemondb.orders (OrderDate, Quantity, TotalCost, OrderType, DeliveryID, MenuID, CustomerID, StaffID)
            VALUES ("{row.OrderDate}", {row.Quantity}, {row.TotalCost}, "{row.OrderType}", {int(row.DeliveryID)}, {row.MenuID}, {row.CustomerID}, {row.StaffID});
            """
        )
    else:
        raise ValueError('Invalid Order Type')

connection.commit()